# How often does @realDonaldTrump tweet each day?

In [1]:
import altair as alt
import pandas as pd
import matplotlib as mpl
import numpy as np
import json
import pytz
from datetime import datetime
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Read dump downloaded from trumptweetarchive.com

In [2]:
%run '00-trump-tweets-processing.ipynb'

In [3]:
src = pd.read_json('/Users/mhustiles/data/github/notebooks/trump-tweet-frequency/output/realdonaldtrump.json') 

### How many tweets since May 2009?

In [4]:
len(src)

55064

In [5]:
src.head()

,index,id,text,isRetweet,isDeleted,device,favorites,retweets,date,eastern_created_at,year,month,day,hour,minute,time
0,0,1329424134166687744,Almost ZERO ballots rejected in Georgia this e...,False,False,Twitter for iPhone,73380,17472,2020-11-19,2020-11-19 13:59:47,2020,11,19,5,59,05:59:47
1,1,1329420741553643520,Thousands of uncounted votes discovered in Geo...,False,False,Twitter for iPhone,90970,20653,2020-11-19,2020-11-19 13:46:18,2020,11,19,5,46,05:46:18
2,2,1329408856733184000,Important News Conference today by lawyers on ...,False,False,Twitter for iPhone,150690,36438,2020-11-19,2020-11-19 12:59:05,2020,11,19,4,59,04:59:05
3,3,1329405682706247680,"Voter Fraud in Detroit is rampant, and has bee...",False,False,Twitter for iPhone,135638,25243,2020-11-19,2020-11-19 12:46:28,2020,11,19,4,46,04:46:28
4,4,1329404785083232256,THE COVID DRUGS NOW AVAILABLE TO MAKE PROPLE B...,False,False,Twitter for iPhone,109129,21291,2020-11-19,2020-11-19 12:42:54,2020,11,19,4,42,04:42:54


### Clean up dates

In [6]:
# src['eastern_created_at'] = src['date'].dt.time
# src['eastern_created_at'] = src['date'].dt.tz_localize("UTC").dt.tz_convert('US/Eastern')
# src['date'] = pd.to_datetime(src['eastern_created_at']).dt.strftime('%m/%d/%Y')
src['year'] = src['eastern_created_at'].dt.year
src['month'] = src['eastern_created_at'].dt.month
src['day'] = src['eastern_created_at'].dt.day
src['hour'] = src['eastern_created_at'].dt.hour
src['minute'] = src['eastern_created_at'].dt.minute
src['time'] = src['eastern_created_at'].dt.time
src['id'] = src['id'].astype(str)
src['year'] = src['year'].astype(str)
src['month'] = src['month'].astype(str)
src['day'] = src['day'].astype(str)
src['hour'] = src['hour'].astype(str)

In [7]:
src.head()

,index,id,text,isRetweet,isDeleted,device,favorites,retweets,date,eastern_created_at,year,month,day,hour,minute,time
0,0,1329424134166687744,Almost ZERO ballots rejected in Georgia this e...,False,False,Twitter for iPhone,73380,17472,2020-11-19,2020-11-19 13:59:47,2020,11,19,13,59,13:59:47
1,1,1329420741553643520,Thousands of uncounted votes discovered in Geo...,False,False,Twitter for iPhone,90970,20653,2020-11-19,2020-11-19 13:46:18,2020,11,19,13,46,13:46:18
2,2,1329408856733184000,Important News Conference today by lawyers on ...,False,False,Twitter for iPhone,150690,36438,2020-11-19,2020-11-19 12:59:05,2020,11,19,12,59,12:59:05
3,3,1329405682706247680,"Voter Fraud in Detroit is rampant, and has bee...",False,False,Twitter for iPhone,135638,25243,2020-11-19,2020-11-19 12:46:28,2020,11,19,12,46,12:46:28
4,4,1329404785083232256,THE COVID DRUGS NOW AVAILABLE TO MAKE PROPLE B...,False,False,Twitter for iPhone,109129,21291,2020-11-19,2020-11-19 12:42:54,2020,11,19,12,42,12:42:54


### Most popular tweets

In [8]:
src[['id','date','text','favorites' ]].sort_values('favorites', ascending=False).head(5)

,id,date,text,favorites
10515,1311892190680014848,2020-10-01,"Tonight, @FLOTUS and I tested positive for COV...",1869706
10563,1312233807991496704,2020-10-02,"Going welI, I think! Thank you to all. LOVE!!!",1219870
386,1325099845045071872,2020-11-07,"I WON THIS ELECTION, BY A LOT!",1194053
329,1326158760826560512,2020-11-10,WE WILL WIN!,1020245
384,1325195021339987968,2020-11-07,"71,000,000 Legal Votes. The most EVER for a si...",983186


In [9]:
src[['id','date','text','retweets']].sort_values('retweets', ascending=False).head(5)

,id,date,text,retweets
10515,1311892190680014848,2020-10-01,"Tonight, @FLOTUS and I tested positive for COV...",408866
34290,881503147168071680,2017-07-02,#FraudNewsCNN #FNN https://t.co/WYUnHjjUjg,293109
37978,795954831718498304,2016-11-08,TODAY WE MAKE AMERICA GREAT AGAIN!,281289
28288,474134260149157888,2014-06-04,Are you allowed to impeach a president for gro...,237674
7847,1267637602724839424,2020-06-01,RT @SpaceX: Liftoff! https://t.co/DRBfdUM7JA,235250


### Since he took office

In [10]:
trumppres = pd.DataFrame(src[src.date >= '01/20/2017']).sort_values('date', ascending=True)

---

### Devices used for tweets as president

In [11]:
trumppres_decices = trumppres.groupby(['device']).agg({'id':'count'}).reset_index().rename(columns={'id': 'count'})

In [12]:
for i in trumppres_decices:
    trumppres_decices['pct'] = round((trumppres_decices['count']/trumppres_decices['count'].sum())*100,2)

In [13]:
trumppres_decices.sort_values('count', ascending=False).head()

,device,count,pct
7,Twitter for iPhone,24185,97.31
2,Twitter Media Studio,367,1.48
5,Twitter for Android,173,0.70
4,Twitter Web Client,48,0.19
6,Twitter for iPad,38,0.15


### How many not from an iPhone since president? 

In [14]:
trumppres_notiphone = trumppres[trumppres['device'] != 'Twitter for iPhone']

In [15]:
len(trumppres_notiphone)

669

### Time difference between tweets

In [16]:
trumppres = pd.DataFrame(src[src.date >= '01/20/2017']).sort_values('date', ascending=True)

In [17]:
trumppres['diff'] = (trumppres['date'] - trumppres['date'].shift(1)).astype('timedelta64[h]')

In [18]:
trumppres.sort_values('diff', ascending=False).head()

,index,id,text,isRetweet,isDeleted,device,favorites,retweets,date,eastern_created_at,year,month,day,hour,minute,time,diff
46695,46601,993476979893723136,"Good luck to Ric Grenell, our new Ambassador t...",f,f,Twitter for iPhone,55387,9326,2018-05-07,2018-05-07 13:05:44,2018,5,7,13,5,13:05:44,48.0
43923,43817,853604334944354304,Our military is building and is rapidly becomi...,f,f,Twitter for iPhone,88055,16425,2017-04-16,2017-04-16 13:41:47,2017,4,16,13,41,13:41:47,48.0
47825,47737,959029384052166656,Heading to beautiful West Virginia to be with ...,f,f,Twitter for iPhone,117593,22624,2018-02-01,2018-02-01 11:43:17,2018,2,1,11,43,11:43:17,48.0
47659,47571,968702480753315840,"Such a beautiful map, thank you! https://t.co/...",f,f,Twitter for iPhone,59188,13004,2018-02-27,2018-02-28 04:20:43,2018,2,28,4,20,04:20:43,48.0
43635,43527,873120139222306816,"Despite so many false statements and lies, tot...",f,f,Twitter for iPhone,104596,29083,2017-06-09,2017-06-09 10:10:37,2017,6,9,10,10,10:10:37,48.0


In [19]:
trumppres['diff'].mean()

1.3509837846537642

### Tweets per month, year

In [20]:
trump_years = src.groupby(['year','month']).agg({'date': 'count'}).reset_index()

In [21]:
trump_years.tail()

,year,month,date
134,2020,5,1100
135,2020,6,1092
136,2020,7,849
137,2020,8,1108
138,2020,9,1346


### What did he average each day in retweets, likes and replies

In [22]:
trump_engagements_day = src.groupby(['month', 'year']).agg({'id':'size', 'retweets':['sum', 'mean'], 'favorites':['sum', 'mean']}).round().reset_index()

In [23]:
trump_engagements_day.columns = trump_engagements_day.columns.map('_'.join).str.strip().str.rstrip('_')

In [24]:
trump_engagements_day.tail()

,month,year,id_size,retweets_sum,retweets_mean,favorites_sum,favorites_mean
134,9,2016,295,2206872,7481.0,5785857,19613.0
135,9,2017,306,3987482,13031.0,16293825,53248.0
136,9,2018,381,5157591,13537.0,19077421,50072.0
137,9,2019,785,9351654,11913.0,30052019,38283.0
138,9,2020,1346,21110233,15684.0,57328518,42592.0


In [25]:
chart = alt.Chart((trump_engagements_day.query('year > "2016"')))\
    .mark_bar(size=10,opacity=1)\
    .encode(
    x=alt.X('month:T', title=' ', \
            axis=alt.Axis(values=['1', '12'], grid=False, tickCount=1, format='%b')),
    y=alt.Y('id_size:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=2, format='1,s'),\
           scale=alt.Scale(domain=(0, 1000))),\
    facet=alt.Facet('year:N', columns=4, title=' ')
).properties(width=150, height=100,
     title='Monthly tweets by @realDonaldTrump'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

In [26]:
chart = alt.Chart((trump_engagements_day.query('year > "2016"')))\
    .mark_bar(size=10,opacity=1)\
    .encode(
    x=alt.X('month:T', title=' ', \
            axis=alt.Axis(values=['1', '12'], grid=False, tickCount=1, format='%b')),
    y=alt.Y('favorites_mean:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=2, format='1,s'),\
           scale=alt.Scale(domain=(0, 120000))),\
    facet=alt.Facet('year:N', columns=4, title=' ')
).properties(width=150, height=100,
     title='Mean monthly likes of @realDonaldTrump'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

In [27]:
chart = alt.Chart((trump_engagements_day.query('year > "2016"')))\
    .mark_bar(size=10,opacity=1)\
    .encode(
    x=alt.X('month:T', title=' ', \
            axis=alt.Axis(values=['1', '7'], grid=False, tickCount=1, format='%B')),
    y=alt.Y('retweets_mean:Q', stack=None, title=' ', axis=alt.Axis(gridWidth=.6,\
     gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=2, format='1,s'),\
           scale=alt.Scale(domain=(0, 30000))),\
    facet=alt.Facet('year:N', columns=4, title=' ')
).properties(width=150, height=100,
     title='Mean monthly retweets of @realDonaldTrump'
)

chart.configure_view(strokeOpacity=0).configure_legend(
    orient='top',
    symbolType='stroke'
).configure_legend(symbolType='square')

alt.Chart(...)

### Limit data to when Trump took office. How many tweets since then?

In [28]:
trumpall_prez = src[src.date >= '01/20/2017']
len(trumpall_prez)

24854

In [29]:
trumpall_prez

,index,id,text,isRetweet,isDeleted,device,favorites,retweets,date,eastern_created_at,year,month,day,hour,minute,time
0,0,1329424134166687744,Almost ZERO ballots rejected in Georgia this e...,False,False,Twitter for iPhone,73380,17472,2020-11-19,2020-11-19 13:59:47,2020,11,19,13,59,13:59:47
1,1,1329420741553643520,Thousands of uncounted votes discovered in Geo...,False,False,Twitter for iPhone,90970,20653,2020-11-19,2020-11-19 13:46:18,2020,11,19,13,46,13:46:18
2,2,1329408856733184000,Important News Conference today by lawyers on ...,False,False,Twitter for iPhone,150690,36438,2020-11-19,2020-11-19 12:59:05,2020,11,19,12,59,12:59:05
3,3,1329405682706247680,"Voter Fraud in Detroit is rampant, and has bee...",False,False,Twitter for iPhone,135638,25243,2020-11-19,2020-11-19 12:46:28,2020,11,19,12,46,12:46:28
4,4,1329404785083232256,THE COVID DRUGS NOW AVAILABLE TO MAKE PROPLE B...,False,False,Twitter for iPhone,109129,21291,2020-11-19,2020-11-19 12:42:54,2020,11,19,12,42,12:42:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55059,55085,1319485303363571712,RT @RandPaul: I don’t know why @JoeBiden think...,t,f,Twitter for iPhone,0,20683,2020-10-22,2020-10-23 03:46:25,2020,10,23,3,46,03:46:25
55060,55086,1319484210101379072,RT @EliseStefanik: President @realDonaldTrump ...,t,f,Twitter for iPhone,0,9869,2020-10-22,2020-10-23 03:42:05,2020,10,23,3,42,03:42:05
55061,55087,1319444420861829120,RT @TeamTrump: LIVE: Presidential Debate #Deba...,t,f,Twitter for iPhone,0,8197,2020-10-22,2020-10-23 01:03:58,2020,10,23,1,3,01:03:58
55062,55088,1319384118849949696,Just signed an order to support the workers of...,f,f,Twitter for iPhone,176289,36001,2020-10-22,2020-10-22 21:04:21,2020,10,22,21,4,21:04:21


### Limit data to month Trump announced candidacy. How many tweets since then?

In [30]:
trumpall_running = src[src.date >= '06/01/2015']
len(trumpall_running)

33238

### Count daily tweets since candidacy, create dataframe with results

In [31]:
trump_tweets_day_candidacy = trumpall_running.groupby(['date']).size()
trump_tweets_day_candidacy_df = pd.DataFrame({'date': trump_tweets_day_candidacy.index, 'count': trump_tweets_day_candidacy.values})

In [32]:
trump_tweets_day_candidacy_df.to_csv('output/trump_tweets_day_candidacy.csv')

### Count daily tweets since inaguration, create dataframe with results

In [33]:
trump_tweets_day_prez = trumpall_prez.groupby(['date']).size()
trump_tweets_day_prez_df = pd.DataFrame({'date': trump_tweets_day_prez.index, 'count': trump_tweets_day_prez.values})

In [34]:
trump_tweets_day_prez_df.to_csv('output/trump_tweets_day_prez.csv')

### Count daily tweets from the beginning, create dataframe with results

In [35]:
trump_tweets_day = src.groupby(['date']).agg('size').reset_index(name='count')
trump_tweets_day.tail(10)

,date,count
3443,2020-11-10,37
3444,2020-11-11,24
3445,2020-11-12,48
3446,2020-11-13,41
3447,2020-11-14,38
3448,2020-11-15,45
3449,2020-11-16,41
3450,2020-11-17,22
3451,2020-11-18,26
3452,2020-11-19,11


In [36]:
trump_tweets_day.to_csv('output/trump_tweets_day.csv')

### Sort table for top tweet days during presidency

In [37]:
trump_freq_inauguration = trump_tweets_day_prez_df.sort_values(['count'], ascending=False)

In [38]:
trump_freq_inauguration.head(5)

,date,count
1225,2020-06-05,198
1199,2020-05-10,124
1049,2019-12-12,122
1354,2020-10-12,117
1311,2020-08-30,109


### Read all from today

In [39]:
today = pd.Timestamp.today().strftime("%Y-%m-%d")

In [40]:
len(today)

10

In [41]:
trump_tweets_today = trumpall_prez[trumpall_prez.date == today]

In [42]:
trump_tweets_today.head()

,index,id,text,isRetweet,isDeleted,device,favorites,retweets,date,eastern_created_at,year,month,day,hour,minute,time
0,0,1329424134166687744,Almost ZERO ballots rejected in Georgia this e...,False,False,Twitter for iPhone,73380,17472,2020-11-19,2020-11-19 13:59:47,2020,11,19,13,59,13:59:47
1,1,1329420741553643520,Thousands of uncounted votes discovered in Geo...,False,False,Twitter for iPhone,90970,20653,2020-11-19,2020-11-19 13:46:18,2020,11,19,13,46,13:46:18
2,2,1329408856733184000,Important News Conference today by lawyers on ...,False,False,Twitter for iPhone,150690,36438,2020-11-19,2020-11-19 12:59:05,2020,11,19,12,59,12:59:05
3,3,1329405682706247680,"Voter Fraud in Detroit is rampant, and has bee...",False,False,Twitter for iPhone,135638,25243,2020-11-19,2020-11-19 12:46:28,2020,11,19,12,46,12:46:28
4,4,1329404785083232256,THE COVID DRUGS NOW AVAILABLE TO MAKE PROPLE B...,False,False,Twitter for iPhone,109129,21291,2020-11-19,2020-11-19 12:42:54,2020,11,19,12,42,12:42:54


---

### RT vs Tweet

In [43]:
rt_tweet = src.groupby(['date', 'isRetweet']).agg('size').reset_index(name='count')
rt_tweet_recent = src[src['date'] > '2019-01-01']\
    .groupby(['date', 'isRetweet']).agg('size').reset_index(name='count')

In [44]:
rt_pivot = pd.DataFrame(pd.pivot_table(rt_tweet, values='count', index=['date'],\
                          columns=['isRetweet'], aggfunc=np.sum).\
                        rename(columns={'isRetweet':'id', False:'tweet', True:'retweet'}).reset_index().fillna(''))

In [45]:
rt_pivot.to_csv('output/rt_pivot.csv')

---

### Chart it!

In [46]:
#bars

lines = alt.Chart(trump_tweets_day_candidacy_df, title = '@realDonaldTrump tweet frequency since candidacy announcement').mark_bar(size=2).encode(
    x = alt.X('date:T', axis = alt.Axis(grid=False, title = '', tickCount=6, format = ("%b. %Y"))),
    y = alt.Y('count:Q',
        scale=alt.Scale(domain=(0, 200)), axis = alt.Axis(gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0, tickCount=6, title = 'Daily tweet counts and mean')),
)

#rule showing mean

rule = alt.Chart(trump_tweets_day_candidacy_df).mark_rule(color='red').encode(
    y='mean(count):Q'
)

#rule label -- would like to add "Average: " annotation
text = rule.mark_text(
    align='center',
    baseline='middle',
    dx=200,
    dy=10,
    fontWeight='bold',
).encode(
    text=alt.Text('mean(count):Q', format=".2"))

#go
( lines + rule + text ).properties(height=400,width=800).configure_view(strokeOpacity=0)

alt.LayerChart(...)

In [47]:
#bars

bars = alt.Chart(trump_freq_inauguration, title = '@realDonaldTrump tweet frequency since inauguration').\
    mark_bar(size=2).encode(
    x = alt.X('date:T', axis = alt.Axis(grid=False, title = '', tickCount=6, format = ("%b. %Y"))),
    y = alt.Y('count:Q',
        scale=alt.Scale(domain=(0, 200)), axis = alt.Axis(gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0, tickCount=6, title = 'Daily tweet counts and mean')),
)

#rule showing mean

rule = alt.Chart(trump_freq_inauguration).mark_rule(color='red').encode(
    y='mean(count):Q'
)

#rule label -- would like to add "Average: " annotation
text = rule.mark_text(
    align='center',
    baseline='middle',
    dx=200,
    dy=10,
    fontWeight='bold',
).encode(
    text=alt.Text('mean(count):Q', format=".2"))

#go
( bars + rule + text ).properties(height=400,width=800).configure_view(strokeOpacity=0)

alt.LayerChart(...)

In [48]:
heatmap = alt.Chart(
    trump_freq_inauguration,
    title="@realDonaldTrump daily tweet frequency since inauguration"
).mark_rect().encode(
    x=alt.X('date(date):O', title=' '),
    y=alt.Y('month(date):O', title=''),
    color=alt.Color('count:Q', title='Tweet count'),
    tooltip=[
        alt.Tooltip('monthdate(date):T', title='Date'),
        alt.Tooltip('count:Q', title='Tweets')
    ],
    facet=alt.Facet('year(date):O', title='',columns=1)
).properties(width=600, height=200)

heatmap.configure_legend(
    orient='top',
    padding=10
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [49]:
trump_RT_freq_inauguration = trump_tweets_day_prez_df.sort_values(['count'], ascending=False)

In [50]:
trumpall_prez_rt = trumpall_prez[(trumpall_prez['isRetweet'] == True) | (trumpall_prez['isRetweet'] == 't')]
trumpall_prez_no_rt = trumpall_prez[(trumpall_prez['isRetweet'] == False) | (trumpall_prez['isRetweet'] == 'f')]

In [51]:
len(trumpall_prez_rt)

9196

### Retweets

In [52]:
trump_tweets_day_prez_rt = trumpall_prez_rt.groupby(['date']).size()
trump_tweets_day_prez_rt_df = pd.DataFrame({'date': trump_tweets_day_prez_rt.index, \
                                         'count': trump_tweets_day_prez_rt.values})

In [53]:
trump_tweets_day_prez_rt_df.sort_values(by='date', ascending=False).head()

,date,count
856,2020-11-19,1
855,2020-11-18,9
854,2020-11-17,5
853,2020-11-16,20
852,2020-11-15,21


### No retweets

In [54]:
trump_tweets_day_prez_no_rt = trumpall_prez_no_rt.groupby(['date']).size()
trump_tweets_day_prez_no_rt_df = pd.DataFrame({'date': trump_tweets_day_prez_no_rt.index, \
                                         'count': trump_tweets_day_prez_no_rt.values})

In [55]:
# retweets
trump_rt_freq_inauguration = trump_tweets_day_prez_rt_df.sort_values(['count'], ascending=False)
# no retweets
trump_no_rt_freq_inauguration = trump_tweets_day_prez_no_rt_df.sort_values(['count'], ascending=False)

In [56]:
trump_rt_freq_inauguration.head()

,date,count
698,2020-06-05,162
672,2020-05-10,104
530,2019-12-12,92
526,2019-12-08,81
781,2020-08-30,78


In [57]:
### Try a different color scheme 

heatmap = alt.Chart(
    trump_no_rt_freq_inauguration,
    title="@realDonaldTrump daily tweet (without RT) frequency since inauguration"
).mark_rect().encode(
    x=alt.X('date(date):O', title=' '),
    y=alt.Y('month(date):O', title=''),
    color=alt.Color('count:Q', title='Tweet count', scale=alt.Scale(scheme="blues")),
    tooltip=[
        alt.Tooltip('monthdate(date):T', title='Date'),
        alt.Tooltip('count:Q', title='Tweets')
    ],
    facet=alt.Facet('year(date):O', title='', columns=1)
).properties(width=600, height=180)

heatmap.configure_legend(
    orient='top',
    padding=10
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [58]:
#bars
bars = alt.Chart(trump_no_rt_freq_inauguration, \
                 title = '@realDonaldTrump tweet frequency (without RT) since inauguration').mark_bar(size=2).encode(
    x = alt.X('date:T', axis = alt.Axis(grid=False, title = '', tickCount=6, format = ("%b. %Y"))),
    y = alt.Y('count:Q',
        scale=alt.Scale(domain=(0, 150)), axis = alt.Axis(gridColor='#dddddd',offset=6,tickSize=0,domainOpacity=0,tickCount=5, title = 'Daily tweet counts and mean')),
)

#rule showing mean
rule = alt.Chart(trump_no_rt_freq_inauguration).mark_rule(color='red').encode(
    y='mean(count):Q'
)

#rule label -- would like to add "Average: " annotation
text = rule.mark_text(
    align='center',
    baseline='middle',
    dx=50,
    dy=10,
    fontWeight='bold',
).encode(
    text=alt.Text('mean(count):Q', format=".2"))

#go
( bars + rule + text ).properties(height=400,width=800).configure_view(strokeOpacity=0)

alt.LayerChart(...)